In [1]:
from google.colab import drive
drive.mount("/content/drive/", force_remount=True)
import os
os.chdir('./drive/My Drive/TESI/Autoencoder')

Mounted at /content/drive/


In [2]:
import pickle as pickle

from src import TaxiBJ, TaxiBJ3d
import numpy as np

np.random.seed(1337)  # for reproducibility

# PARAMETERS
model_name = 'model3'
DATAPATH = '../data'  
CACHEDATA = True  # cache data or NOT
nb_epoch = 50 # number of epoch at training stage
# nb_epoch_cont =  100 # number of epoch at training (cont) stage
batch_size = 16  # batch size
T = 48  # number of time intervals in one day
lr = 0.00015 # learning rate

len_closeness = 3 # length of closeness dependent sequence
len_period = 1 # length of peroid dependent sequence
len_trend = 1 # length of trend dependent sequence
nb_flow = 2  # there are two types of flows: inflow and outflow
# divide data into two subsets: Train & Test, of which the test set is the
# last 4 weeks
days_test = 7*4
len_test = T*days_test
len_val = 2*len_test
map_height, map_width = 32, 32  # grid size

path_log = 'log_BJ'
muilt_step = False

cache_folder = 'Autoencoder/model3' if model_name == 'model3' else 'Autoencoder'
path_cache = os.path.join(DATAPATH, 'CACHE', cache_folder)  # cache path
path_result = 'RET'
path_model = 'MODEL'
if os.path.isdir(path_result) is False:
    os.mkdir(path_result)
if os.path.isdir(path_model) is False:
    os.mkdir(path_model)
if CACHEDATA and os.path.isdir(path_cache) is False:
    os.mkdir(path_cache)

In [3]:
from utils import cache, read_cache
import time
import os

# load data
if muilt_step:
    dic_rmse={}
    list_muilt_rmse=[]
print("loading data...")
ts = time.time()
fname = os.path.join(path_cache, 'TaxiBJ_C{}_P{}_T{}.h5'.format(
    len_closeness, len_period, len_trend))
if os.path.exists(fname) and CACHEDATA:
    X_train_all, Y_train_all, X_train, Y_train, \
    X_val, Y_val, X_test, Y_test, mmn, external_dim, \
    timestamp_train_all, timestamp_train, timestamp_val, timestamp_test = read_cache(
        fname, 'preprocessing_bj.pkl')
    print("load %s successfully" % fname)
else:
    if (model_name == 'model3'):
        load_data = TaxiBJ3d.load_data
    else:
        load_data = TaxiBJ.load_data
    X_train_all, Y_train_all, X_train, Y_train, \
    X_val, Y_val, X_test, Y_test, mmn, external_dim, \
    timestamp_train_all, timestamp_train, timestamp_val, timestamp_test = load_data(
        T=T, nb_flow=nb_flow, len_closeness=len_closeness, len_period=len_period, len_trend=len_trend, len_test=len_test,
        len_val=len_val, preprocess_name='preprocessing_bj.pkl', meta_data=True, meteorol_data=True, holiday_data=True, datapath=DATAPATH)
    if CACHEDATA:
        cache(fname, X_train_all, Y_train_all, X_train, Y_train, X_val, Y_val, X_test, Y_test,
              external_dim, timestamp_train_all, timestamp_train, timestamp_val, timestamp_test)
i = 0
print(external_dim)
print("\n days (test): ", [v[:8] for v in timestamp_test[0::T]])
print("\nelapsed time (loading data): %.3f seconds\n" % (time.time() - ts))


loading data...
file name:  ../data/TaxiBJ/BJ13_M32x32_T30_InOut.h5


/content/drive/My Drive/TESI/Autoencoder/src/__init__.py:29: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  with h5py.File(fname) as f:
/content/drive/My Drive/TESI/Autoencoder/src/__init__.py:32: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  mmax = f['data'].value.max()
/content/drive/My Drive/TESI/Autoencoder/src/__init__.py:33: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  mmin = f['data'].value.min()


=====stat=====
data shape: (4888, 2, 32, 32)
# of days: 121, from 2013-07-01 to 2013-10-29
# of timeslots: 5808
# of timeslots (available): 4888
missing ratio of timeslots: 15.8%
max: 1230.000, min: 0.000
=====stat=====
incomplete days:  [b'20130926']


file name:  ../data/TaxiBJ/BJ14_M32x32_T30_InOut.h5


/content/drive/My Drive/TESI/Autoencoder/src/__init__.py:11: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  data = f['data'].value
/content/drive/My Drive/TESI/Autoencoder/src/__init__.py:12: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  timestamps = f['date'].value


=====stat=====
data shape: (4780, 2, 32, 32)
# of days: 119, from 2014-03-01 to 2014-06-27
# of timeslots: 5712
# of timeslots (available): 4780
missing ratio of timeslots: 16.3%
max: 1292.000, min: 0.000
=====stat=====
incomplete days:  [b'20140304', b'20140313', b'20140323', b'20140326', b'20140401', b'20140402', b'20140409', b'20140410', b'20140412', b'20140422', b'20140501', b'20140526', b'20140618', b'20140627']


file name:  ../data/TaxiBJ/BJ15_M32x32_T30_InOut.h5
=====stat=====
data shape: (5596, 2, 32, 32)
# of days: 122, from 2015-03-01 to 2015-06-30
# of timeslots: 5856
# of timeslots (available): 5596
missing ratio of timeslots: 4.4%
max: 1274.000, min: 0.000
=====stat=====
incomplete days:  [b'20150320', b'20150404', b'20150620']


file name:  ../data/TaxiBJ/BJ16_M32x32_T30_InOut.h5
=====stat=====
data shape: (7220, 2, 32, 32)
# of days: 162, from 2015-11-01 to 2016-04-10
# of timeslots: 7776
# of timeslots (available): 7220
missing ratio of timeslots: 7.2%
max: 1250.000, m

/content/drive/My Drive/TESI/Autoencoder/src/TaxiBJ3d.py:45: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  Timeslot = f['date'].value
/content/drive/My Drive/TESI/Autoencoder/src/TaxiBJ3d.py:46: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  WindSpeed = f['WindSpeed'].value
/content/drive/My Drive/TESI/Autoencoder/src/TaxiBJ3d.py:47: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  Weather = f['Weather'].value
/content/drive/My Drive/TESI/Autoencoder/src/TaxiBJ3d.py:48: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  Temperature = f['Temperature'].value


shape:  (15072,) (15072, 17) (15072,)
time feature: (15072, 8) holiday feature: (15072, 1) meteorol feature:  (15072, 19) mete feature:  (15072, 28)
(13728, 5, 32, 32, 2)
(13728, 28)

(12384, 5, 32, 32, 2)
(12384, 28)

(1344, 5, 32, 32, 2)
(1344, 28)

(1344, 5, 32, 32, 2)
(1344, 28)

28

 days (test):  [b'20160310', b'20160311', b'20160312', b'20160313', b'20160314', b'20160315', b'20160316', b'20160317', b'20160318', b'20160319', b'20160320', b'20160321', b'20160322', b'20160325', b'20160326', b'20160327', b'20160328', b'20160329', b'20160331', b'20160401', b'20160402', b'20160403', b'20160404', b'20160405', b'20160406', b'20160407', b'20160408', b'20160409']

elapsed time (loading data): 77.727 seconds



In [4]:
from keras.callbacks import EarlyStopping, ModelCheckpoint
from utils import build_model

model = build_model(len_closeness, len_period, len_trend, model=model_name,
                    external_dim=external_dim, lr=lr,
                    save_model_pic=f'TaxiBJ_{model_name}')
# model.summary()
hyperparams_name = '{}.TaxiBJ.c{}.p{}.t{}.lr{}'.format(
    model_name, len_closeness, len_period, len_trend, lr)
fname_param = os.path.join('MODEL', '{}.best.h5'.format(hyperparams_name))

early_stopping = EarlyStopping(monitor='val_rmse', patience=10, mode='min')
model_checkpoint = ModelCheckpoint(
    fname_param, monitor='val_rmse', verbose=0, save_best_only=True, mode='min')

In [5]:
# train model
print("training model...")
ts = time.time()
history = model.fit(X_train, Y_train,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    validation_data=(X_val,Y_val),
                    callbacks=[early_stopping, model_checkpoint],
                    verbose=2)
model.save_weights(os.path.join(
    'MODEL', '{}.h5'.format(hyperparams_name)), overwrite=True)
pickle.dump((history.history), open(os.path.join(
    path_result, '{}.history.pkl'.format(hyperparams_name)), 'wb'))
print("\nelapsed time (training): %.3f seconds\n" % (time.time() - ts))

training model...
Epoch 1/50
774/774 - 312s - loss: 0.2296 - rmse: 0.4588 - val_loss: 0.1384 - val_rmse: 0.3721
Epoch 2/50
774/774 - 332s - loss: 0.1356 - rmse: 0.3682 - val_loss: 0.1326 - val_rmse: 0.3641
Epoch 3/50
774/774 - 340s - loss: 0.0947 - rmse: 0.3063 - val_loss: 0.0834 - val_rmse: 0.2887
Epoch 4/50
774/774 - 328s - loss: 0.0834 - rmse: 0.2887 - val_loss: 0.0830 - val_rmse: 0.2880
Epoch 5/50
774/774 - 327s - loss: 0.0830 - rmse: 0.2880 - val_loss: 0.0829 - val_rmse: 0.2880
Epoch 6/50
774/774 - 327s - loss: 0.0605 - rmse: 0.2405 - val_loss: 0.0302 - val_rmse: 0.1738
Epoch 7/50
774/774 - 322s - loss: 0.0304 - rmse: 0.1745 - val_loss: 0.0299 - val_rmse: 0.1728
Epoch 8/50
774/774 - 324s - loss: 0.0302 - rmse: 0.1738 - val_loss: 0.0298 - val_rmse: 0.1726
Epoch 9/50
774/774 - 323s - loss: 0.0062 - rmse: 0.0616 - val_loss: 9.3377e-04 - val_rmse: 0.0295
Epoch 10/50
774/774 - 322s - loss: 0.0013 - rmse: 0.0349 - val_loss: 8.7902e-04 - val_rmse: 0.0285
Epoch 11/50
774/774 - 321s - loss

In [6]:
# model.load_weights('MODEL/TaxiBJ.c3.p1.t1.lr0.00015.best.h5')
model.load_weights(fname_param)
score = model.evaluate(
    X_test, Y_test, batch_size=Y_test.shape[0], verbose=0)
print('Test score: %.6f rmse (norm): %.6f rmse (real): %.6f' %
        (score[0], score[1], score[1] * (mmn._max - mmn._min) / 2.))

Test score: 0.000783 rmse (norm): 0.027988 rmse (real): 18.079984
